                                                    mysql server connection

In [2]:
import pymysql.cursors
from IPython.display import clear_output # to clear output of the notebook for better ui


# password value will be removed , enter your own mysql server password to test the code
db =pymysql.connect(host='localhost',user='root',password='',
cursorclass=pymysql.cursors.DictCursor)

                                                    Arrays for each table's records

In [3]:
auhtors=[
    (1,'jake paul','j@net-22330','PHD in science fiction litruture'),
    (2,'sara jay','ss@gmail-4566','a noble winner writer'),
    (3,'andrew tate','andrew@lo-andrewT.com','professional heart doctor'),
    (4,'hamid han','hamid@dr- Drhamid.net','PHD in science'),
    (5,'ahmed tawfiq','23113232','owner of apple company')
]

users=[
    (12,'Hamza Sallam','hamza@gmail'),
    (13,'Yaman Arar','yaman@net'),
    (14,'Mhd Khaifa','mhd@root'),
    (15,'James Bond','james123@apple'),
    (16,'Lucy Waine','waine@tech')
]

users_tel=[
    (12,'123132'),
    (13,'112442'),
    (14,'54353'),
    (15,'454342'),
    (12,'234567')
]

advertises=[
    (101,'apple','technology'),
    (102,'nike','fitness'),
    (103,'wego','travel'),
    (104,'safari','education'),
    (105,'nectar','health')
]

articles=[
    (22,'Ice and fire','2021-3-12','nature',1,104),
    (23,'beuty of Rome city','2019-12-12','travel',2,103),
    (24,'Science and life, How to be a scientest','2021-4-11','science',4,104),
    (25,'GPU vs CPU','2021-01-11','technology',5,105),
    (26,'How to eat junk food and stay healthy','2019-3-12','health and diet',3,102)
]

readsR=[
    (12,22,2.5),
    (12,23,1.5),
    (13,22,5.5),
    (14,23,2.5),
    (15,26,1.5),
    (16,22,2.2),
    (15,23,0.5)
]


                                            Creation and population of the Tables


In [4]:

with db.cursor() as c: 
    try:
        c.execute("USE NEWSPAPER")
    except pymysql.Error:    
        c.execute("CREATE DATABASE NEWSPAPER;")
        c.execute("USE NEWSPAPER;")
    #author table   
        c.execute("""CREATE TABLE author(
            author_id INT, 
            name VARCHAR(45),
            contact VARCHAR(45),
            bio VARCHAR(45),
            PRIMARY KEY(author_id)
        );""")
    # user table
        c.execute("""CREATE TABLE user(
        user_id INT,
        name VARCHAR(45),
        email VARCHAR(45),
        PRIMARY KEY(user_id)        
        );""") 
    #user-telephone table
        c.execute("""CREATE TABLE user_tel( 
        user_id INT,
        telephone VARCHAR(45),
        PRIMARY KEY(user_id,telephone),
        FOREIGN KEY(user_id) REFERENCES user(user_id)
        );""") 
    #advertise table:
        c.execute("""CREATE TABLE advertise(
        Ad_id INT,
        company VARCHAR(45),
        type VARCHAR(45),
        PRIMARY KEY(Ad_id)        
        );""") 
    #article table
        c.execute("""CREATE TABLE article(
        article_id INT, 
        title VARCHAR(45),
        publication_date DATE,
        category VARCHAR(45),
        author_id INT,
        AD_id INT, 
        PRIMARY KEY(article_id),
        FOREIGN KEY(author_id) REFERENCES author(author_id),
        FOREIGN KEY(AD_id) REFERENCES advertise(Ad_id)
        );""")
    #READS table:
        c.execute("""CREATE TABLE read_s(
        user_id INT,
        article_id INT,
        hours_spent FLOAT,
        PRIMARY KEY(user_id,article_id),
        FOREIGN KEY(user_id) REFERENCES user(user_id),
        FOREIGN KEY(article_id) REFERENCES article(article_id)        
        );""")
#****************************************Populate the table using the records array*********************************************
        c.executemany("insert into author values(%s,%s,%s,%s);",auhtors)
        c.executemany("insert into user values(%s,%s,%s);",users)
        c.executemany("insert into user_tel values(%s,%s);",users_tel)
        c.executemany("insert into advertise values(%s,%s,%s);",advertises)
        c.executemany("insert into article values(%s,%s,%s,%s,%s,%s);",articles)
        c.executemany("insert into read_s values(%s,%s,%s);",readsR)
        db.commit()  #commit the insertion 

                                helper function to check if table asked from user exist or not

In [5]:
def tableExist(table):
    #list of the tables of the database
    tables=['author','article','user','advertise','read_s','user_tel']
    if table in tables:
        return True
    else:
        return False

                                        helper method to display tables in a good looking way  

In [6]:
def print_table(table):
  # Get the list of keys for the columns
  keys = table[0].keys()

  # Find the longest value for each key
  column_widths = []
  for key in keys:
    max_length = max([len(str(d[key])) for d in table])
    column_widths.append(max(len(key), max_length))
  
  # Print the table header
  header = "|"
  for i, key in enumerate(keys):
    header += " " + key.upper().ljust(column_widths[i]) + " |"
  print(header)
  print("-" * len(header))
  
  # Print the table rows
  for t in table:
    row = "|"
    for i, key in enumerate(keys):
      row += " " + str(t[key]).ljust(column_widths[i]) + " |"
    print(row)


                                    helper method to assign an array of attributes of asked table

In [7]:
def attributesOF(table):
    tables=['author','article','user','advertise','read_s','user_tel']
    if table == tables[0]:
        attrs=['author_id','name','contact','bio']
    if table == tables[1]:
        attrs=['article_id','title','publication_date','category','author_id','AD_id']    
    if table == tables[2]:
        attrs=['user_id','name','email']  
    if table == tables[3]:
        attrs=['Ad_id','company','type']  
    if table == tables[4]:
        attrs=['user_id','article_id','hours_spent']
    if table == tables[5]:
        attrs=['user_id','telephone']
    return attrs 

                                            a function to display records

In [8]:
def showRecords():
    c = db.cursor()
    table=input("enter the table you want to see: press 'e' to exit")
    while True:
        if tableExist(table):
            command=f"select * from {table};"
            break
        if table == 'e':
            return
        table=input("table not found, try again: press 'e' to exit")
    c.execute(command)
    # call the method to display the table
    print_table(c.fetchall())
    
    input(f" {table} table displayed ... press enter to go back to main menu")    

    

                                        a function to add records from user's input 

In [9]:
def addRecords():
    table= input("enter the table you want to add a record to: press 'e' to exit\n")
    while True:
        if tableExist(table):
            sqlcommand=f"insert into {table} values("
            break
        if table == 'e':
            return
        table=input("table not found, try again: press 'e' to exit")
    #call attributesOf to get the attributs of asked table
    attrs= attributesOF(table)
    for attr in attrs:
        value= input(f"Enter {attr}")
        sqlcommand+=f"'{value}',"

    #delete the extra comma 
    sqlcommand = f"{sqlcommand[:-1]});"
    try:
        db.cursor().execute(sqlcommand)
        print("the record is added successfully")
        db.commit()
    except pymysql.Error as err:
        print(err)        
    input(f"you successfully added a new record to {table} table... press enter to go back to main menu")    



                                                    Complex SQL commands Section

a function that execute complex sql command no 1 : checks the author's name of an article published in a certin year

In [10]:
def checkAuthorsName():
    c= db.cursor()
    year = input("enter the year")
    try:
        c.execute(f""" 
        SELECT author.name FROM article
        INNER JOIN author
        ON author.author_id = article.author_id
        WHERE YEAR(article.publication_date) = '{year}'
        ;""")
        print_table(c.fetchall())
    except pymysql.Error as err:
        print(f" incorrect input for year, please try again\n{err}")

a function that execute complex sql command no 2: show adverisment info of the article

In [11]:
def checkAdInfo():
    c= db.cursor()
    article=input("enter article id: ")
    try:
        c.execute(f"""
        SELECT advertise.* FROM advertise 
        INNER JOIN article
        ON advertise.Ad_id = article.Ad_id
        ;""")
        print_table(c.fetchall())
    except pymysql.Error as err:
        print("incorrect article id , please try again\n err")      

a function that execute complex sql command no 3 : print article info and author's name which users spent more than a given hours on it

In [12]:
def ArticleInfo():
    c=db.cursor()
    hours=input("enter number of hours:  ")
    try:
        c.execute(f"""
        SELECT article.title, article.publication_date,article.category,author.name as "author's_name"  FROM article
        INNER JOIN read_s
        ON article.article_id = read_s.article_id
        INNER JOIN author
        ON article.author_id=author.author_id
        WHERE read_s.hours_spent >= {hours}
        ;""")
        print_table(c.fetchall())#print the query

    except pymysql.Error as err:
        print(f"invalid input please try again\n{err}")   

a function to let the user enter the complex sql command himself

In [13]:
def CustomSqlStatment():
    c=db.cursor()
    command = input("enter your own sql command: \n->:")
    try:
        c.execute(command)
        print_table(c.fetchall())
    except pymysql.Error as err:
        print(f"the sql command is wrong , please write it correctly\n{err}")

function where the user choose which complex sql statment to use

In [14]:
def chooseComplexSql():
    while True:
        clear_output()
        options=['1','2','3','4','5']
        print("""choose a command to execute:\n\n
        1-check the author's name of an article published in a certin year\n
        2- show adverisment info of the article\n
        3-print article info and author's name which users spent more than a given hours on it\n
        4- write your own sql statment\n
        5- Exit\n 
        """)
        option = input("answer: ")
        if option not in options:
            option =input("invalid option, try again")
        if option =="1":
            checkAuthorsName()
            break
        if option == "2":
            checkAdInfo() 
            break
        if option =="3": 
            ArticleInfo()
            break
        if option =="4": 
            CustomSqlStatment()
            break
        if option =="5":
            return
    input(f"you successfully executed a complex sql command,..press enter to go back to main menu")        

                            Additional functionality (BONUS):  a function to delete records from a table

In [15]:
def deleteRecords():
    c=db.cursor()
    table=input("Enter table name you want to delete from: press 'e' to exit")
    while True:
        if(tableExist(table)):
            command  = f'delete from {table} where '
            break
        if table == 'e': #if user clicks 'e' exit the function
            return
        table=input("incorrect table name, please try again")
    # to delete from a many-to-many relation , the composite primary keys must be asked
    if table == 'read_s' or table== 'user_tel':
        id1=attributesOF(table)[0]
        id2=attributesOF(table)[1]
        Id1selected= input(f"enter {id1}")
        Id2selected= input(f"enter {id2}")
        command+=f"{id1} = {Id1selected} and {id2} = {Id2selected};"    
    else:    
    #get the id of the table by calling attributseOF method and getting the first attribute which is the id
        id=attributesOF(table)[0]
        Idselected= input(f"enter {id} to delete: ")
        command+=f"{id} = {Idselected};"
    try:
        c.execute(command)
        print(f"successfully deleted a record from the {table} table ")
        db.commit()
    except pymysql.Error as err:
        print(f"no id is found , please try again\n{err}")

    input("you successfully asked to delete a record... press enter to go back to main menu")    



                                                    main method : 

In [16]:
def main():
    answers =['a','b','c','d','e'] #options user choose to apply functions
    db.commit() #commit any changes during the run
    while True:
        clear_output() # clear the output when user goes back to main menu
        print(""" WELCOME to the Istanbul post DataBase\n please choose an option to apply functions:\n
    a- show records of a table
    b- add records to a table
    c- apply complex queries
    d- delete a record from a table
    e- exit
    """)
        answer = input("answer:")
        if answer not in answers:
            print("invalid option")
            answer = input("invalid  option, try again")
        if answer =="a":
            showRecords()
        if answer == "b":
            addRecords()
        if answer =="c":
            chooseComplexSql()
        if answer =="d":
            deleteRecords()    
        if answer =="e":
            break               


start the loop 

start the loop 

In [17]:
main()

 WELCOME to the Istanbul post DataBase
 please choose an option to apply functions:

    a- show records of a table
    b- add records to a table
    c- apply complex queries
    d- delete a record from a table
    e- exit
    
